### Clonning source code

In [ ]:
! git clone https://github.com/amaranto/chatbot || ( cd chatbot/ && git pull origin main)

### Mounting ChromaDB

In [ ]:
# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

data_zip = '1ZT0fkKeYiiImxB0HHfFnsYjYDy2wYd8Z' 

download = drive.CreateFile({'id': data_zip})
download.GetContentFile('data.zip')

In [ ]:
! unzip data.zip 
! mv data/ chatbot/src/

### Install requirements

In [ ]:
! pip install chromadb sentence_transformers SPARQLWrapper

### Set Hugging Face

If you want to use hugging face to run generator model you can set HFACE_TOKEN environment var in next step.
If it is not defined, it will colab cpu or gpu if it is available 

In [ ]:
! export HFACE_TOKEN='YOUR TOKEN' && export ENV='colab'

### Example to use ChromaManager
Using chromadb already copied in previous step.
You can point to a remote chroma db if you want providing url, port and password.
In addition, there is Chroma docker file to build in your local for local development.

gen_vector_db.py is an example about how to generate a chroma collection for this bot.

In [ ]:
! export ENV='colab' && cd chatbot/src && python3 gen_vector_db.py

### Example of chatbot usage
chat.py is an example about how to use ChatBot module.

It requires a ChromaManager db class to work in order to search for embeddings to generate context about specific topics.
In this example we will be using the local chroma db cloned with the repository.
But you point ChromaManager to an external database with another topics.


In [ ]:
! export HFACE_TOKEN='HTOKEN' && export ENV='colab' && cd chatbot/src && python3 chat.py